In [ ]:
## Package Import
import sys
import os 
import numpy as np
import pandas as pd
from datetime import datetime
import re
import matplotlib.pyplot as plt

In [ ]:
!gsutil cp $WORKSPACE_BUCKET/data/gwas_v2/phenotypes/celiac_demo_v2.tsv .

In [ ]:
samples = pd.read_csv('hla_types_with_DQ.csv')

In [ ]:
#we first calculated HLA-types 50 random samples
1st_ran = samples.sample(50)

In [ ]:
#then, gather those with no risk allele and celiac disease 
ced = samples[(samples['genotype']=='X/X')&(samples['condition']=='celiac')].reset_index(drop=True)
ced.to_csv('hladq_XX_and_CeD.csv',index=False)

In [ ]:
#also 200 with dq2/dq8,half patient half control
risk_test=samples[samples['genotype'].str.contains('DQ2|DQ8')&(samples['condition']=='control')&~samples['sample.id'].isin(hla_types['person_id'])].sample(100)
non_risk_test=samples[~samples['genotype'].str.contains('DQ2|DQ8')&(samples['condition']=='control')&~samples['sample.id'].isin(hla_types['person_id'])].sample(100)
test_2 = pd.concat([risk_test,non_risk_test])

In [ ]:
total=pd.concat([1st_ran,ced,test_2])

In [ ]:
#cram manifest files
!gsutil -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v7/wgs/cram/manifest.csv .
cram_manifest=pd.read_csv('manifest.csv')
cram_manifest_v7 =cram_manifest[cram_manifest['person_id'].astype(str).isin(total["person_id"])]

In [ ]:
cram_manifest_v7.reset_index(drop=True,inplace=True)

cram_manifest_v7['cram_uri'].to_csv('hla_to_type.txt',index=False, header=None)

#for reproducibility, we have a list in our workspace bucket containing the exact samples we have typed in our workspace bucket; these will be called in the next notebook